In [ ]:
!pip install geemap
!pip install psycopg2
!pip install folium
!pip install pycrs

In [ ]:
import geemap
import ee
# from geemap import geojson_to_ee
import datetime as datetime
import numpy as np
import json
from scipy.stats import norm
from dateutil.relativedelta import relativedelta
import sys

In [ ]:
#Earth Engine Initialization

ee.Authenticate()
ee.Initialize(project='anaghamurali')

Map = geemap.Map()

In [ ]:
#defining roi
roi = ee.Geometry.Polygon(
        [[[78.15721339392177, 21.93703133269448],
          [78.15721339392177, 20.657559259898125],
          [79.90403956579677, 20.657559259898125],
          [79.90403956579677, 21.93703133269448]]]);
chirps_data = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterBounds(roi)
# # print(chirps_data.size().getInfo())

In [ ]:
# roi = ee.FeatureCollection('projects/anaghamurali/assets/Climate_zone_1').filterMetadata('Zone', "equals",'Composite');
# chirps_data = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterBounds(roi)
# print(chirps_data.size().getInfo())

In [ ]:
# #defining roi
# roi = ee.FeatureCollection('projects/ee-nitish1996/assets/Mumbai').filterMetadata('DISTRICT_L', "equals",'483')
# chirps_data = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterBounds(roi)
# # print(chirps_data.size().getInfo())

In [ ]:
# defining time range
start_date = ee.Date('2001-01-01')
end_date = ee.Date('2022-12-31')

start = datetime.datetime(2001,1,1)
end = datetime.datetime(2022,12,31)

step = 1 # monthly, 3months, 6months


months = (end.year-start.year) * 12 + (end.month-start.month) +1
# print('Months:',months)

count = int((months/step)+1)
# print('count:',count)

count_list = np.arange(count)
# print(count_list)

month_lst = []
for i in count_list:
    month = start
    delta = i * step
    month = month + relativedelta(months = delta)
    print(month)
    month_lst.append(month)

print(month_lst) #date_list

In [ ]:
#calculating the sum of daily precipitation according to the scale
i = 0
chirps_lst = []
n = len(month_lst)
while i < n:
  if i+1<n:
    start_1 = month_lst[i]
    end_1 = month_lst[i+1]
    chirps_filtered = chirps_data.filterDate(start_1,end_1).sort('system:time_start', False).filterBounds(roi).select("precipitation")
    chirps_sum = chirps_filtered.sum().clip(roi)

    chirps_start_date = chirps_filtered.first().get('system:time_start')
    chirps_end_date = ee.Date(chirps_filtered.limit(1,'system:time_end', False).first().get('system:time_end'))
    chirps_sum_1 = chirps_sum.set({
                      'Start_Date': chirps_start_date,
                      'End_Date': chirps_end_date,
                      'system:time_start': chirps_filtered.first().get('system:time_start'),
                      'system:time_end' : chirps_filtered.limit(1,'system:time_end', False).first().get('system:time_end')})
  else:
    break

  chirps_lst.append(chirps_sum_1)
  i = i+1

In [ ]:
chirps_lst

In [ ]:
# Increase the recursion limit (use with caution)
sys.setrecursionlimit(15000)

In [ ]:
# calculation og SPI for the different datasets
for n in range(0,len(chirps_lst)):

  img = chirps_lst[n]                                                   # assigning img to variable
  values = img.reduceRegion(reducer=ee.Reducer.toList(), geometry=roi, scale=5550)    #converting the data into a list
  values_arr  = values.toArray()                                        # converting data into array
  values_list = values_arr.getInfo()
  values_flatten = np.array(values_list).flatten()                      # converting data into 1d array
  values_sorted = np.sort(values_flatten)                               # sorting the data

  rank_arr = np.arange(1,len(values_sorted)+1)                          # rank of the sorted array

  class NpEncoder(json.JSONEncoder):                                    # to account for JSON error
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

  # Your data containing NumPy arrays
  data = {
    'array1': values_sorted,
    'array2': rank_arr,
  }

  # Convert NumPy arrays to Python lists before serializing to JSON
  for key, value in data.items():
    if isinstance(value, np.ndarray):
        data[key] = value.tolist()

  # Convert the data to JSON
  json_data = json.dumps(data, cls=NpEncoder)

  # Print the JSON data
  # print(json_data)

  zero = ee.Image.constant(0).clip(roi)                                     # SPI calculation
  for i in range(len(values_sorted)):
    const = ee.Image.constant(1).clip(roi)
    value = data['array1'][i]
    rank = data['array2'][i]
    val = (rank - 0.44)/(len(values_sorted)+0.12)
    spi = ee.Number(norm.ppf(val))
    mask = img.where(img.neq(value),0)
    masking = const.multiply(mask)
    masked = masking.where(masking.neq(0),spi)
    zero = zero.add(masked)


# saving the data in drive
  k = month_lst[n]
  task = ee.batch.Export.image.toDrive(image=zero,
                                     description='Image_Export',
                                     folder='GEE_Export_comp',
                                     fileNamePrefix='SPI_'+ str(k),
                                     region=roi,
                                     scale=5550)
  task.start()

KeyboardInterrupt: 

In [ ]:
#maping
map = geemap.Map()
map.addLayer(roi,{},'SPI')
map

In [ ]:
#maping
map = geemap.Map()
map.addLayer(zero,{'min':-4,'max':4},'SPI')
map

In [ ]:
ET_data = ee.ImageCollection("MODIS/061/MOD16A2").filterBounds(roi)

In [ ]:
# defining time range
start_date = ee.Date('2001-01-01')
end_date = ee.Date('2022-12-31')

start = datetime.datetime(2001,1,1)
end = datetime.datetime(2020,12,31)

step = 1 # monthly, 3months, 6months


months = (end.year-start.year) * 12 + (end.month-start.month) +1
# print('Months:',months)

count = int((months/step)+1)
# print('count:',count)

count_list = np.arange(count)
# print(count_list)

month_lst = []
for i in count_list:
    month = start
    delta = i * step
    month = month + relativedelta(months = delta)
    print(month)
    month_lst.append(month)

print(month_lst) #date_list

In [ ]:
#calculating the sum of daily precipitation according to the scale
i = 0
chirps_lst = []
n = len(month_lst)
while i < n:
  if i+1<n:
    start_1 = month_lst[i]
    end_1 = month_lst[i+1]
    chirps_filtered = ET_data.filterDate(start_1,end_1).sort('system:time_start', False).filterBounds(roi).select("precipitation")
    chirps_sum = chirps_filtered.sum().clip(roi)

    chirps_start_date = chirps_filtered.first().get('system:time_start')
    chirps_end_date = ee.Date(chirps_filtered.limit(1,'system:time_end', False).first().get('system:time_end'))
    chirps_sum_1 = chirps_sum.set({
                      'Start_Date': chirps_start_date,
                      'End_Date': chirps_end_date,
                      'system:time_start': chirps_filtered.first().get('system:time_start'),
                      'system:time_end' : chirps_filtered.limit(1,'system:time_end', False).first().get('system:time_end')})
  else:
    break

  chirps_lst.append(chirps_sum_1)
  i = i+1

In [ ]:
spei = ee.ImageCollection("CSIC/SPEI/2_8").filterBounds(roi).select('SPEI_03_month')

In [ ]:
def clip_image(image):
    return image.clip(roi)

In [ ]:
clippedCollection = spei.map(clip_image)

In [ ]:
#maping
map = geemap.Map()
map.addLayer(clippedCollection,{'min':-4,'max':4},'SPEI')
map

In [ ]:
map = geemap.Map()
map.addLayer(clippedCollection,{'min':-4,'max':4},'SPEI')
map.addLayer(zero,{'min':-4,'max':4},'SPI')
map

In [ ]:
ET_data = ee.ImageCollection("MODIS/006/MOD16A2").filterBounds(roi)

In [ ]:
# defining time range
start_date = ee.Date('2001-01-01')
end_date = ee.Date('2022-12-31')

start = datetime.datetime(2001,1,1)
end = datetime.datetime(2022,12,31)

step = 3 # monthly, 3months, 6months


months = (end.year-start.year) * 12 + (end.month-start.month) +1
# print('Months:',months)

count = int((months/step)+1)
# print('count:',count)

count_list = np.arange(count)
# print(count_list)

month_lst = []
for i in count_list:
    month = start
    delta = i * step
    month = month + relativedelta(months = delta)
    print(month)
    month_lst.append(month)

print(month_lst) #date_list

In [ ]:
#calculating the sum of daily precipitation according to the scale
i = 0
ET_lst = []
n = len(month_lst)
while i < n:
  if i+1<n:
    start_1 = month_lst[i]
    end_1 = month_lst[i+1]
    ET_filtered = ET_data.filterDate(start_1,end_1).sort('system:time_start', False).filterBounds(roi).select("ET")
    ET_sum = ET_filtered.sum().clip(roi)

    ET_start_date = ET_filtered.first().get('system:time_start')
    ET_end_date = ee.Date(ET_filtered.limit(1, 'system:time_end', False).first().get('system:time_end'))
    ET_sum_1 = ET_sum.set({
                      'Start_Date': ET_start_date,
                      'End_Date': ET_end_date,
                      'system:time_start': ET_filtered.first().get('system:time_start'),
                      'system:time_end' : ET_filtered.limit(1,'system:time_end', False).first().get('system:time_end')})
  else:
    break

  ET_lst.append(ET_sum_1)
  i = i+1

In [ ]:
ET_lst

In [ ]:
# Increase the recursion limit (use with caution)
sys.setrecursionlimit(5000)

In [ ]:
# calculation og SPI for the different datasets
for n in range(0,len(ET_lst)):

  img = ET_lst[n]                                                   # assigning img to variable
  values = img.reduceRegion(reducer=ee.Reducer.toList(), geometry=roi, scale=5550)    #converting the data into a list
  values_arr  = values.toArray()                                        # converting data into array
  values_list = values_arr.getInfo()
  values_flatten = np.array(values_list).flatten()                      # converting data into 1d array
  values_sorted = np.sort(values_flatten)                               # sorting the data

  rank_arr = np.arange(1,len(values_sorted)+1)                          # rank of the sorted array

  class NpEncoder(json.JSONEncoder):                                    # to account for JSON error
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

  # Your data containing NumPy arrays
  data = {
    'array1': values_sorted,
    'array2': rank_arr,
  }

  # Convert NumPy arrays to Python lists before serializing to JSON
  for key, value in data.items():
    if isinstance(value, np.ndarray):
        data[key] = value.tolist()

  # Convert the data to JSON
  json_data = json.dumps(data, cls=NpEncoder)

  # Print the JSON data
  # print(json_data)

  zero = ee.Image.constant(0).clip(roi)                                     # SPI calculation
  for i in range(len(values_sorted)):
    const = ee.Image.constant(1).clip(roi)
    value = data['array1'][i]
    rank = data['array2'][i]
    val = (rank - 0.44)/(len(values_sorted)+0.12)
    ET = ee.Number(norm.ppf(val))
    mask = img.where(img.neq(value),0)
    masking = const.multiply(mask)
    masked = masking.where(masking.neq(0),ET)
    zero = zero.add(masked)


# saving the data in drive
  k = month_lst[n]
  task = ee.batch.Export.image.toDrive(image=zero,
                                     description='Image_Export',
                                     folder='GEE_Export',
                                     fileNamePrefix='ET_'+ str(k),
                                     region=roi,
                                     scale=1000)
  task.start()

In [ ]:
#maping
map = geemap.Map()
map.addLayer(zero,{'min':-4,'max':4},'SPEI')
map

In [ ]:
    # const = ee.Image.constant(1).clip(roi)
    # value = ee.Number(data['array1'][i])
    # rank = ee.Number(data['array2'][i])
    # len_values = ee.Number(len(values_sorted))
    # val = (rank.subtract(0.44)).divide(len_values.add(0.12))
    # ET = val.erfInv()
    # mask = img.where(img.neq(value),0)
    # masking = const.multiply(mask)
    # masked = masking.where(masking.neq(0),ET)
    # zero = zero.add(masked)